## Introduction to Chemical Hamiltonians
### What problem are we trying to solve?
Given a set of nuclei (classical point charges) surrounded by a cloud of quantum electrons, we want to find the arrangement of electrons around the fixed nuclei such that the energy of the system is minimised.

### Quantum mechanical model of the system
Due to their small size, the motion of electrons around the nuclei is governed by the Schrodinger equation, which is a linear partial differential equation of the form

$$H_{el}\Psi=E\Psi$$
where $$\Psi = \Psi(\vec{r_1}, \omega_1, \vec{r_2}, \omega_2,...) = \Psi(\vec{x_1}, \vec{x_2},...) \text{ : represents the state of electrons}$$

$$\hat{H}_{el} = \underset{\text{One-Electron Operators}}{\underset{\text{Electronic K.E.}}{\hat{T}_e(r)}  + \underset{\text{Elec/Nuc attraction}}{\hat{V}_{eN}(r; R)}} + \underset{\text{Two-Electron Operator}}{\underset{\text{Elec/Elec repulsion}}{\hat{V}_{ee}(r)}} = \underset{\text{One-Electron Operator}}{\sum_{i}\hat{h}(i)} + \underset{\text{Two-Electron Operator}}{\sum_{i\ne j}\hat{v}(i, j)}$$

Using $i$ and $j$ to index electrons, and $A$ and $B$ to index nuclei, we have (in atomic units)
$$
\hat{H}_{el} = - \sum_i \frac{1}{2} \nabla^2_i - \sum_{Ai} \frac{Z_A}{r_{Ai}} + \sum_{i>j} \frac{1}{r_{ij}}
$$

Our goal is to find the lowest energy states of the electrons subject to this hamiltonian.

### Discretizing the representation of the system
The wave function $\Psi(x)$ is an infinite dimensional object, and to represent it on a computer, we need to make the representation of the system finite. In order to do this, we introduce a set of 1-electron basis functions called orbitals.

$$ \text{Finite Basis: } \{ \chi_1, \chi_2, \dots \chi_m \} $$
such that given any wavefunction $\Psi(x)$, we can approximately write it in this basis of 1 electron basis functions.

### Second quantization representation
Because of Pauli Exclusion Principle, we know that no two electrons can have the stame state. Therefore, instead of asking the question *which electron is in which orbital*, in second quantization representation we ask the question *how many electrons are present in an orbital*. 

Therefore, the state of a system with $n$ electrons and $m$ basis functions can be represented by a superposition of occupation basis vectors 
$$\text{Occupation Basis: } \{|n\rangle : n \in \{0,1\}^m, \sum_{p}{n_p} = n\}$$

The second quantization formalism has creation ($a_p^\dagger$) and annihilation ($a_p$) operators to add or remove particles from the occupation number vector.

### Molecular Hamiltonian in Second Quantization
Given a basis set $\{\chi_{M}(x)\}$, the electronic molecular Hamiltonian in second quantization is given as 
$$
\hat{H}_{el} = \sum_{PQ}h_{PQ}a_{P}^\dagger a_{Q} + \frac{1}{2}\sum_{PQRS}g_{PQRS}a_{P}^\dagger a_{R}^\dagger a_{S} a_{Q}
$$


The coefficients $h_{PQ}$ and $g_{PQRS}$ depend on the molecule and set of basis functions chosen. These coefficients are precomputed for a number of molecules and common chemical basis sets and can be easily obtained using quantum computational chemistry packages like OpenFermion.


Thus, the second quantization representation gives us a discrete representation of the molecular Hamiltonian as a sum of tensor products.

## Introduction To OpenFermion
OpenFermion is a quantum computational chemistry package that allows for the easy representation and manipulation of operators on strongly-correlated fermionic, bosonic, and qubit systems. Beginning with an interface to common
electronic structure packages, it simplifies the translation between a molecular specification and a quantum circuit for solving or studying the electronic structure problem on a quantum computer, minimizing the amount of domain expertise required to enter the field.

In [1]:
#Install Openfermion
!pip3 install openfermion --quiet

## Load Molecular Data For $H_2$
### The `MolecularData` Class
OpenFermion uses the `MolecularData` class to store data from computational  chemistry packages. This class has additionally a load and save function, as this data often takes a significant amount of time to prepare. To load stored data, we need to state the geometry of the molecule (in cartesian co-ordinates), the basis in which the molecule is stored, and its charge and multiplicity (i.e. $2n+1$, where $n$ is the number of unpaired spins). OpenFermion also requests a description of the molecule studied

Let's load the molecular data for H2, which is already provided with the package.

In [2]:
import openfermion
diatomic_bond_length = .7414

geometry = [('H', (0., 0., 0.)), 
            ('H', (0., 0., diatomic_bond_length))]

basis = 'sto-3g'
multiplicity = 1
charge = 0
description = format(diatomic_bond_length)

molecule = openfermion.MolecularData(
    geometry,
    basis,
    multiplicity,
    description=description)

# Load already saved molecular data
molecule.load()

print(molecule.name)

H2_sto-3g_singlet_0.7414


In addition to the above data, and being able to generate the Hamiltonian of the target molecule, the MolecularData class contains various information about classical computational approximations of the ground state energy.



In [3]:
print("Bond Length in Angstroms: {}".format(diatomic_bond_length))
print("Hartree Fock (mean-field) energy in Hartrees: {}".format(molecule.hf_energy))
print("FCI (Exact) energy in Hartrees: {}".format(molecule.fci_energy))

Bond Length in Angstroms: 0.7414
Hartree Fock (mean-field) energy in Hartrees: -1.116684387085341
FCI (Exact) energy in Hartrees: -1.137270174660903


## Populate MolecularData by running prominent chemistry packages like Psi4 and PySCF
Openfermion supports prominent electronic structure packages Psi4 and PySCF and provides plugin libraries to interact with these external packages.

### OpenFermion-Psi4
The module run_psi4.py provides a user-friendly way of running Psi4 calculations in OpenFermion. The basic idea is that once one generates a MolecularData instance, one can then call Psi4 with a specification of certain options (for instance, how much memory to use and what calculations to do) in order to compute things about the molecule, update the MolecularData object, and save results of the calculation. For more information, please see [openfermionpsi4_demo.ipynb](https://github.com/quantumlib/OpenFermion-Psi4/blob/master/examples/openfermionpsi4_demo.ipynb)


As an example, we can use Psi4 to compute the HF and FCI energies of a molecule by specifying parameters as shown below.

In [4]:
#@title Install Psi4 and Openfermion-Psi4
!conda install psi4 --yes
!pip3 install openfermionpsi4 --quiet

Solving environment: done

# All requested packages already installed.



In [5]:
from openfermionpsi4 import run_psi4

molecule = openfermion.MolecularData(
    geometry,
    basis,
    multiplicity,
    description=description)

# Run Psi4
molecule = run_psi4(molecule,
                    run_scf=True,
                    run_mp2=False,
                    run_cisd=False,
                    run_ccsd=False,
                    run_fci=True)

print("Bond Length in Angstroms: {}".format(diatomic_bond_length))
print("Hartree Fock (mean-field) energy in Hartrees: {}".format(molecule.hf_energy))
print("FCI (Exact) energy in Hartrees: {}".format(molecule.fci_energy))

Bond Length in Angstroms: 0.7414
Hartree Fock (mean-field) energy in Hartrees: -1.1166843870661929
FCI (Exact) energy in Hartrees: -1.1372701746571041


### OpenFermion-PySCF
The module run_pyscf.py provides a user-friendly way of running PySCF calculations in OpenFermion. The basic idea is that once one generates a MolecularData instance, one can then call PySCF with a specification of certain options (for instance, how much memory to use and what calculations to do) in order to compute things about the molecule, update the MolecularData object, and save results of the calculation. For more information, please see [openfermionpyscf_demo.ipynb](https://github.com/quantumlib/OpenFermion-PySCF/blob/master/examples/openfermionpyscf_demo.ipynb)


As an example, we can use PySCF to compute the HF and FCI energies of a molecule by specifying parameters as shown below.

In [6]:
!pip3 install pyscf --quiet
!pip3 install openfermionpyscf --quiet

In [7]:
from openfermionpyscf import run_pyscf

molecule = openfermion.MolecularData(
    geometry,
    basis,
    multiplicity,
    description=description)

# Run PySCF
molecule = run_pyscf(molecule,
                    run_scf=True,
                    run_mp2=False,
                    run_cisd=False,
                    run_ccsd=False,
                    run_fci=True)

print("Bond Length in Angstroms: {}".format(diatomic_bond_length))
print("Hartree Fock (mean-field) energy in Hartrees: {}".format(molecule.hf_energy))
print("FCI (Exact) energy in Hartrees: {}".format(molecule.fci_energy))

Bond Length in Angstroms: 0.7414
Hartree Fock (mean-field) energy in Hartrees: -1.116684387085341
FCI (Exact) energy in Hartrees: -1.1372701746609035


## Get Molecular Hamiltonian For $H_2$
### The `InteractionOperator` Class
The `InteractionOperator` class provides funcionality to efficiently store and manipulate the molecular hamiltonian as 2- and 4- index tensors corresponding to one- and two- electron interaction terms in the hamiltonian. 

Interaction Operators can be obtained by calling `MolecularData.get_molecular_hamiltonian()`


In [8]:
hamiltonian = molecule.get_molecular_hamiltonian()
print(hamiltonian)

() 0.7137539936876182
((0, 1), (0, 0)) -1.2524635735648988
((1, 1), (1, 0)) -1.2524635735648988
((2, 1), (2, 0)) -0.47594871522096416
((3, 1), (3, 0)) -0.47594871522096416
((0, 1), (0, 1), (0, 0), (0, 0)) 0.3372443831784192
((0, 1), (0, 1), (2, 0), (2, 0)) 0.09064440410574806
((0, 1), (1, 1), (1, 0), (0, 0)) 0.3372443831784192
((0, 1), (1, 1), (3, 0), (2, 0)) 0.09064440410574806
((0, 1), (2, 1), (0, 0), (2, 0)) 0.09064440410574806
((0, 1), (2, 1), (2, 0), (0, 0)) 0.33173404821178437
((0, 1), (3, 1), (1, 0), (2, 0)) 0.09064440410574806
((0, 1), (3, 1), (3, 0), (0, 0)) 0.33173404821178437
((1, 1), (0, 1), (0, 0), (1, 0)) 0.3372443831784192
((1, 1), (0, 1), (2, 0), (3, 0)) 0.09064440410574806
((1, 1), (1, 1), (1, 0), (1, 0)) 0.3372443831784192
((1, 1), (1, 1), (3, 0), (3, 0)) 0.09064440410574806
((1, 1), (2, 1), (0, 0), (3, 0)) 0.09064440410574806
((1, 1), (2, 1), (2, 0), (1, 0)) 0.33173404821178437
((1, 1), (3, 1), (1, 0), (3, 0)) 0.09064440410574806
((1, 1), (3, 1), (3, 0), (1, 0)) 0.33

## Convert Fermionic Molecular Hamiltonian to 4-Qubit Hamiltonian
### The `FermionOperator` Class
The `FermionOperator` class is used to represent the operators on fermionic systems as a sum of creation and annihilation operators. In order to support fast addition of FermionOperator instances, the class is implemented as hash table (python dictionary). The keys of the dictionary encode the strings of ladder operators and values of the dictionary store the coefficients. Each ladder operator is represented by a 2-tuple. The first element of the 2-tuple is an int indicating the tensor factor on which the ladder operator acts. The second element of the 2-tuple is Bool: 1 represents creation and 0 represents annihilation. 

### The `QubitOperator` Class
The `QubitOperator` class is used to store linear combinations of Pauli operators on $N$ qubits. Internally, the data about the operator is stored as a dictionary, with the names of individual Pauli operators used as keys.

### Transformations Between `FermionOperator` and `QubitOperator`
OpenFermion provides utiliites for common transformations like *Jordan-Wigner* and *Bravyi-Kitaev* in order to convert a `FermionOperator` to `QubitOperator`

In [9]:
qubit_hamiltonian = openfermion.jordan_wigner(openfermion.get_fermion_operator(hamiltonian))
print(qubit_hamiltonian)

(-0.09886396933545777+0j) [] +
(-0.045322202052874024+0j) [X0 X1 Y2 Y3] +
(0.045322202052874024+0j) [X0 Y1 Y2 X3] +
(0.045322202052874024+0j) [Y0 X1 X2 Y3] +
(-0.045322202052874024+0j) [Y0 Y1 X2 X3] +
(0.1711977490343295+0j) [Z0] +
(0.1686221915892096+0j) [Z0 Z1] +
(0.12054482205301814+0j) [Z0 Z2] +
(0.16586702410589216+0j) [Z0 Z3] +
(0.17119774903432947+0j) [Z1] +
(0.16586702410589216+0j) [Z1 Z2] +
(0.12054482205301814+0j) [Z1 Z3] +
(-0.22278593040418507+0j) [Z2] +
(0.17434844185575687+0j) [Z2 Z3] +
(-0.2227859304041851+0j) [Z3]


## Next steps (to be contd.)
Once we have the Hamiltonian of the sytem represented as a sum of Pauli strings, the next step is to find a way to (approximately) prepare the ground states of this hamiltonian on a quantum computer. There are various techniques which can be applied to solve this problem and some of these will be discussed in the next tutorial of this series. 

# Appendix
## Introduction to Electronic Structure Theory
Electronic structure theory describes the motions of electrons in atoms or molecules. Because the electrons are so small, one needs to use quantum mechanics to solve for their motion.
### Schrodinger Equation For Molecules
$$H\Psi=E\Psi$$
$$\Psi = \Psi(\vec{r_1}, \omega_1, \vec{r_2}, \omega_2,...) = \Psi(\vec{x_1}, \vec{x_2},...)$$

* The state of an electron is represented by a wavefunction $\Psi(\vec{x})$ where $\vec{x} = (\vec{r}, \omega)$; $\vec{r} = (x, y, z) = (r, \theta, \phi)$ represents the position of the electron and $\omega$ represents represents the spin of the electron.  
* Note that elementary particles like electrons carry an intrinsic form of angular momentum called spin, which cannot be described by a function of positional variables $\vec{r}$, and is somewhat analogous to earth rotating around its own axis.
* Since electrons (fermions) are indistinguishable, the wavefunction $\Psi(\vec{x_1}, \vec{x_2}, ...)$ must be anti-symmetric to satisfy Pauli's Exclusion Principle.

### Molecular Hamiltonian
$$\hat{H} = \underset{\text{Nuclear K.E.}}{\hat{T}_N(R)} + \underset{\text{Electronic K.E.}}{\hat{T}_e(r)} + \underset{\text{Elec/Nuc attraction}}{\hat{V}_{eN}(r, R)} + \underset{\text{Nuc/Nuc repulsion}}{\hat{V}_{NN}(R)} + \underset{\text{Elec/Elec repulsion}}{\hat{V}_{ee}(r)}$$

### Born-Oppenheimer Approximation
* Since nuclei are much larger than electrons, consider the nuclei to be fixed classical point charges. 
* Therefore, nuclear coordinates $R$ become fixed parameters; i.e. $\hat{T}_N(R) \approx 0$, $\hat{V}_{NN}(R)$ is a constant and $\hat{V}_{eN}(r; R)$ depends only parametrically on $R$.
* This results in electronic structure Hamiltonian:

$$\hat{H}_{el} = \underset{\text{One-Electron Operators}}{\underset{\text{Electronic K.E.}}{\hat{T}_e(r)}  + \underset{\text{Elec/Nuc attraction}}{\hat{V}_{eN}(r; R)}} + \underset{\text{Two-Electron Operator}}{\underset{\text{Elec/Elec repulsion}}{\hat{V}_{ee}(r)}} = \underset{\text{One-Electron Operator}}{\sum_{i}\hat{h}(i)} + \underset{\text{Two-Electron Operator}}{\sum_{i\ne j}\hat{v}(i, j)}$$

### Potential Energy Surfaces
* For a given configuration of the nuclei (fixed $R$), we aim to find the the energy eigenstates $|E_i\rangle$ and the corresponding energy eigenvalues $E_i$ of the hamiltonian $H_e$.
* We can solve this equation for a range of nuclear configurations($R$) to map out the potential energy surface of the molecule.

### Orbitals and Chemical Basis Sets
$$\underset{\text{Spin Orbital}}{\chi(\vec{x})} = \underset{\text{Atomic Orbital}}{\phi(\vec{r})} \cdot \underset{\text{Spin Function}}{\sigma(\omega)} $$
* A spin orbital is a one-electron wavefunction which describes the position and spin of an electron in an atom. The spatial component of this one-electron function is called atomic orbital.
* A basis set is a set of functions, called basis functions, such that any elecronic wavefunction can be (approximately) represented as a linear combination of anti-symmetrized product of basis functions. 
* Chemical basis sets are usually composed of spin orbital functions, examples of which include Gaussian-type Orbitals (GTO), Slater-type Orbitals (STO), Slater type orbital-n Gaussians (STO-nG) etc. 

### Slater Determinant
* A Slater determinant is an anti-symmetrized product of one or more spin orbitals. For example, Let $\{\chi_{P}(x)\}$ be a basis of M orthonormal spin orbitals, a normalized Slater determinant for N electrons can be written as:

$$
\begin{eqnarray}
	\left|\chi_{P_1},\chi_{P_2},...,\chi_{P_N}\right| &=& \frac{1}{\sqrt{N!}} 
		\left|
			\begin{array}{cccc}
				\chi_{P_1}(\vec{x_1}) & \chi_{P_2}(\vec{x_1}) & \ldots & \chi_{P_N}(\vec{x_1}) \\
				\chi_{P_1}(\vec{x_2}) & \chi_{P_2}(\vec{x_2}) & \ldots & \chi_{P_N}(\vec{x_2}) \\
				\vdots            & \vdots            & \ddots & \vdots            \\
				\chi_{P_1}(\vec{x_N}) & \chi_{P_2}(\vec{x_N}) & \ldots & \chi_{P_N}(\vec{x_N}) \\
			\end{array}
		\right|
\end{eqnarray}
$$

### Second Quantization / Occupation Number Represenation

* We can also represent the Slater determinant by an *occupation-number vector* $|k\rangle$ s.t.

$$ 
|k\rangle =|k_1, k_2, \ldots, k_M\rangle, k_P = 
\begin{cases}
    1, \  \chi_{P}\ \text{occupied i.e. present in slater determinant}\\
    0, \  \chi_{P}\ \text{unoccupied i.e. absent in slater determinant}
\end{cases}
 $$

* This representation is also called second quantization representation, where instead of asking the question "*Which electron is in which state?*", we ask the question "*How many particles are there in each state?*".

* The second quantization formalism has creation ($a_p^\dagger$) and annihilation ($a_p$) operators to add or remove particles from the many-body electronic wavefunction while maintaining the anti-symmetric property. The action of the operators on determinants is given by
$$
\begin{split} 
a_{p}^{\dagger}|\cdots,k_{p-1},\ k_{p},\ k_{p+1},\cdots\rangle &= (-1)^{\sum_{i < p}k_i}\ (1-&k_p) &|\cdots,k_{p-1},\ 1-k_p,\ k_{p+1},\cdots\rangle \\   
a_{p}|\cdots,k_{p-1},\ k_{p},\ k_{p+1},\cdots\rangle &= (-1)^{\sum _{i < p}k_i}& k_p &|\cdots,k_{p-1},\ 1-k_{p},\ k_{p+1},\cdots\rangle
\end{split}
$$

### Molecular Hamiltonian in Second Quantization
* *One-/Two- electron operators* are of the form $\sum_{i}\hat{h}(i)$ / $\sum_{i\ne j}\hat{v}(i, j)$ such that every individual term can only affect one/two electrons by changing their orbitals with some probability. 
 
* Therefore, given a basis set $\{\chi_{M}(x)\}$, the electronic molecular Hamiltonian in second quantization is given as 
$$
\hat{H}_{el} = \sum_{PQ}h_{PQ}a_{P}^\dagger a_{Q} + \frac{1}{2}\sum_{PQRS}g_{PQRS}a_{P}^\dagger a_{R}^\dagger a_{S} a_{Q}
$$
* Applied to an electronic state, the Hamiltonian produces a linear combination of the original state with states generated by single and double electron excitations from this state. With each such excitation, there is an associated amplitude $h_{PQ}$ or $g_{PQRS}$, which represents the probability
of this event happening. These probability amplitudes are calculated from the spin orbitals $\{\chi_{M}(x)\}$ and the one- and two-electron operators $\sum_{i}\hat{h}(i)$ and $\sum_{i\ne j}\hat{v}(i, j)$.
